# Praktična programska pitanja vezana uz skupove podataka

## 1. Zadatak
Skup podataka „Estonia“ sadrži zemlju podrijetla, ime, spol (M - muški, F - ženski),
godine, kategoriju (P - putnik, C - posada) i sudbinu (0 - poginuo, 1 - preživio) za 742
putnika na brodu MS Estonia u noći potonuća. Za ovaj skup podataka (estoniapassenger-list_modified.csv) odredite:

a) (1 bod) Kako biste klasificirali zadani izvor podataka prema strukturi?


**Odgovor:** 

b) (1 bod) Objasnite o kojoj se vrsti istraživanja prema vrsti korištenih podataka
ovdje radi.

**Odgovor:** 

c) (1 bod) Objasnite radi li se ovdje o studiji presjeka ili o longitudinalnom
istraživanju.

**Odgovor:** 

d) (2 boda) Kako je u ovo istraživanje uključen slučajan podskup od 742 putnika (od
ukupnog broja od 989 putnika), radi li se o slučajnom, pristranom ili neslučajnom
uzorku? Objasnite svoj odgovor. Definirajte svaki od navedenih vrsta uzoraka.

**Odgovor:** 

e) (2 boda) Izbacite značajke “PassenderId”, “Firstname” i “Lastname”. Za svaku od
značajki “Country”, “Sex” i “Category” iskoristite odgovarajuću metodu
pretvorbe (engl. encoder) kako biste kategoričke podatke transformirali u
numeričke vrijednosti.

**Odgovor:** 

f) (2 boda) Provedite zamjenu nedostajućih vrijednosti značajke “Age” mjerom
očuvanja sredine. Pojasnite koju mjeru sredine koristite i zašto.

**Odgovor:**

g) (2 boda) Provjerite prikladnim prikazom je li distribucija značajke “Age”
simetrična. Ako nije, objasnite o kojoj se vrsti asimetričnosti radi.

**Odgovor:**

h) (4 boda) Provedite postupak DBSCAN kako biste otkrili stršeće vrijednosti u ovom
skupu podataka ne uzimajući u obzir ciljnu značajku (“Survived”). Po potrebi
varirajte parametre algoritma te na temelju odgovora algoritma provjerite
naznačene pojedince. Navedite redne brojeve stršećih podataka.

**Odgovor:**

(3 boda) Provjerite mogu li se stršeće vrijednosti otkriti koristeći samo metode
vizualizacije podataka. Pritom primijenite prikladni graf za njihovo otkrivanje.
Koji graf koji ste isprobali daje najbolji prikaz stršećih vrijednosti?i) 

**Odgovor:**

j) (4 boda) Korištenjem prikladnog algoritma nadziranog strojnog učenja s jasnim
tumačenjem ustanovite koja zemlja je bila najsretnija pri preživljavanju. Navedite
o kojoj zemlji se radi i gdje u modelu se ta činjenica vidi. Pritom model učite na
slučajno izdvojenih 67% podataka, a testirajte na preostalom skupu.

**Odgovor:**

## 2. Zadatak

Skup podataka „Possum“ sadrži podatke o 104 planinska četkasta oposuma pronađena
na sedam lokacija u Australiji: redni broj oposuma, oznaku lokacije gdje je pronađen
oposum, populaciju (Vic – Victoria, other – Novi Južni Wales ili Queensland), spol (M -
muški, F – ženski), dob, duljina glave u mm, širina lubanje u mm, ukupna duljina u cm,
duljina repa u cm, duljina stopala u mm, duljina ušne školjke u mm, udaljenost od
medijalnog kantusa do lateralnog kantusa desnog oka u mm, obujam prsa u cm i opseg
trbuha u cm. Za ovaj skup podataka (possum.csv) odredite:

a) (3 boda) Navedite metode prikupljanja podataka. Koja od navedenih metoda je
mogla biti upotrijebljena prilikom stvaranja ovog skupa podataka? Objasnite.

b) (3 boda) Kako možemo ukloniti statistički redundantne (nebitne) značajke?
Primjenom odgovarajuće metode utvrdite postoje li takve numeričke značajke u
ovom skupu i uklonite ih.

c) (4 boda) Za preostale numeričke značajke iz prethodnog dijela zadatka provedite
standardizaciju z-vrijednosti (z-skor) te potom primijenite grupiranje postupkom
k-srednjih vrijednosti (engl. k-means, uz variranje hiperparametra k između 1 i 5).
Pritom prikažite grupiranja primjeraka u dvije dimenzije za parove varijabli iz
skupa “eye“, “chest“ i “belly“. Uočavaju li se grupe? Na što treba obratiti
pozornost pri primjeni algoritma k-srednjih vrijednosti?

d) (2 boda) Izračunajte srednju vrijednost, standardnu devicijaciju i interkvartalno
raspršenje za sve numeričke značajke u ovom skupu. Vizualizirajte ih na istom
grafu koristeći kutijasti graf (engl. box plot).

e) (5 bodova) Modelirajte varijablu “totlngth” na temelju ostalih numeričkih
varijabli u skupu koristeći višestruku linearnu regresiju pri čemu je prethodno
potrebno standardizirati prediktorske varijable. Prikažite rezultate modela. Koje
su pretpostavke korištenja linearnog regresijskog modela?

f) (2 boda) Definirajte srednju kvadratnu pogrešku i primijenite ju na model iz
prethodnog pitanja koji se ispituje na izdvojenom testnom skupu – slučajnih 30%
primjeraka skupa.

## 3. Zadatak

EuroSAT (eurosat.zip) je skup satelitskih RGB slika dimenzija 64x64 koje su raspoređene
u deset klasa ovisno o tome što je na njima prikazano (urbana područja, šume, polja, …).
Vaš zadatak je složiti model dubokog učenja i naučiti ga na danom skupu podataka.
Prikažite implementirani kod.

a) (1 bod) Jedan od prvih koraka u pripremi slika za duboko učenje je njihova
anotacija. Objasnite što je anotirano u danom skupu slika.

b) (2 boda) Učitajte dani skup podataka i podijelite ga na skup za učenje i skup za
testiranje. Za samo učitavanje podataka možete iskoristiti već implementirani
razred `torch.utils.data.Dataset`.

In [ ]:
class EuroSATDataset(Dataset):
  def __init__(self, data_dir):
    self.instances = []

    curr_label = -1
    for subdir, _, files in os.walk(data_dir):
      if curr_label < 0:
        curr_label += 1
        continue

      for file in files:
        if file.endswith('.jpg'):
          self.instances.append((os.path.join(subdir, file), curr_label))
      curr_label += 1

  def __len__(self):
    return len(self.instances)

  def __getitem__(self, idx):
    image = read_image(self.instances[idx][0]) / 255.0
    label = self.instances[idx][1]

    return image, label

In [ ]:
dataset = EuroSATDataset("./data_dir")

c) (3 boda) Složite model (bilo kakav) kojim ćete izvršiti klasifikaciju danih slika.

d) (3 boda) Prikažite matricu zabune. Izračunajte točnost, preciznost, odziv (recall) i
F1-mjeru (engl. F1-score). Objasnite dobivene rezultate i uspješnost modela
kojeg ste upotrijebili.

e) (5 bodova) Napišite funkciju za učenje modela i naučite implementirani model
(dovoljno je da model naučite na samo jednoj epohi => num_epochs=1). Ispišite
iznos funkcije gubitka i točnost vašeg modela na testnom skupu podataka.

# Dodatna teorijska pitanja

## 4. Zadatak

Marko je pri učenju primjetio da mu se model dosta lagano prenauči i stoga je
odlučio uvesti određene metode regularizacije. Točnije, odlučio je koristiti dropout i to
na takav način da bi poredao više dropout slojeva jedan za drugim:

In [ ]:
def Model(nn.Module):
    def __init__(self, . . .):
        # ... other code ...
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.15)
        self.dropout3 = nn.Dropout(p=0.2)
        # ... other code
    def forward(self, x):
        # ... other code ...
        x = self.dropout1(x)
        x = self.dropout2(x)
        x = self.dropout3(x)
        # ... other code ...

Ima li ovakvo redanje dropout slojeva smisla? Je li moguće postići isti regularizacijski
učinak s manje linija koda? Obrazložite svoj odgovor i modificirajte Markov kod!

## 5. Zadatak

Osim primjene metode očuvanja mjere sredine, koje ste još postupke mogli primijeniti za zamjenu nedostajuće vrijednosti nekom drugom u 1. zadatku?

**Odgovor:**

## 6. Zadatak

Objasnite princip učenja značajki (engl. feature learning, representation learning) u kontekstu ekspertnih značajki i ulaznih podataka.

**Odgovor:**

## 7. Zadatak

Objasnite koja je razlika između reziduala i pogreške kod regresije. 

**Odgovor:**

## 8. Zadatak

 Što testiramo hi-kvadrat testom, kakve frekvencije koristimo u izračunu hikvadrat vrijednosti?

**Odgovor:**

## 9. Zadatak

Za algoritam k-NN, navedite kako izabrati k u praksi.

**Odgovor:**

## 10. Zadatak

Navedite koja svojstva funkcija mora zadovoljavati da bude mjera udaljenosti.

**Odgovor:**

## 11. Zadatak

Navedite barem dva slučaja kada biste primijenili umjetno povećavanje broja
primjera u skupu podataka.

**Odgovor:**